In [ ]:
import pandas as pd
import joblib
from sklearn.linear_model import LogisticRegression
from datasets import load_dataset

dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")
df = dataset['train'].to_pandas()
dftest=pd.DataFrame(dataset['test'])
dfval=pd.DataFrame(dataset['validation'])

In [ ]:
print(df.head())
print(dftest.head())
print(dfval.head())

      id                                               text  label sentiment
0   9536                    Cooking microwave pizzas, yummy      2  positive
1   6135  Any plans of allowing sub tasks to show up in ...      1   neutral
2  17697   I love the humor, I just reworded it. Like sa...      2  positive
3  14182                       naw idk what ur talkin about      1   neutral
4  17840          That sucks to hear. I hate days like that      0  negative
      id                                               text  label sentiment
0   9235                         getting cds ready for tour      1   neutral
1  16790   MC, happy mother`s day to your mom ;).. love yah      2  positive
2  24840  A year from now is graduation....i am pretty s...      0  negative
3  20744              because you had chips and sale w/o me      1   neutral
4   6414          Great for organising my work life balance      2  positive
      id                                               text  label sentiment

In [22]:
#irrelevent columns
drop_cols = [
    'id',
    'sentiment'
]

df.drop(columns=drop_cols, inplace=True)
dftest.drop(columns=drop_cols, inplace=True)
dfval.drop(columns=drop_cols, inplace=True)

In [23]:
df = df.rename(columns={"label":"target"})
dftest = dftest.rename(columns={"label":"target"})
dfval = dfval.rename(columns={"label":"target"})

In [24]:
print( df.head())
print( dftest.head())
print( dfval.head())

                                                text  target
0                    Cooking microwave pizzas, yummy       2
1  Any plans of allowing sub tasks to show up in ...       1
2   I love the humor, I just reworded it. Like sa...       2
3                       naw idk what ur talkin about       1
4          That sucks to hear. I hate days like that       0
                                                text  target
0                         getting cds ready for tour       1
1   MC, happy mother`s day to your mom ;).. love yah       2
2  A year from now is graduation....i am pretty s...       0
3              because you had chips and sale w/o me       1
4          Great for organising my work life balance       2
                                                text  target
0  Laying in bed til workkk... Oh the life. Defin...       0
1   ooohhh imma need you to get on that asap love...       2
2   Thanks! I love it they have a video, so you d...       2
3     I left my ipod in 

In [25]:
df["target"].unique()
dftest[ "target"].unique()
dfval["target"].unique()

array([0, 2, 1])

In [26]:
df.to_csv("az_train_review.csv",index = False )
dftest.to_csv("az_test_review.csv",index = False )
dfval.to_csv("az_val_review.csv",index = False )

In [27]:
import re
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm

tqdm.pandas()
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("english"))

URL_PATTERN = re.compile(r"https?://\S+|www\.\S+")
MENTION_PATTERN = re.compile(r"@\w+")
HASHTAG_PATTERN = re.compile(r"#(\w+)")
RT_PATTERN = re.compile(r"\brt\b", re.IGNORECASE)
NON_ALNUM_PATTERN = re.compile(r"[^a-z0-9\s\.\,\!\?\']")


def Amazon(text: str) -> str:
    if not isinstance(text, str):
        return ""
    t = text.strip()

    # Remove URLs and mentions
    t = URL_PATTERN.sub(" ", t)
    
    t = MENTION_PATTERN.sub(" ", t)

    # Keep hashtag words (drop '#')
    t = HASHTAG_PATTERN.sub(r"\1", t)


    # Remove RT markers
    t = RT_PATTERN.sub(" ", t)

    # Lowercase
    t = t.lower()


    # Remove special characters (keep basic punctuation)
    t = NON_ALNUM_PATTERN.sub(" ", t)

    # Normalize whitespace
    t = re.sub(r"\s+", " ", t).strip()

    # Remove stopwords (light)
    tokens = [w for w in t.split() if w not in STOPWORDS]
    t = " ".join(tokens)

    return t

if __name__ == "__main__":
    df = pd.read_csv("az_train_review.csv")
    df["clean_text"] = df["text"].progress_apply(Amazon)
if __name__ == "__main__":
    df1= pd.read_csv("az_test_review.csv")
    df1["clean_text"] = df1["text"].progress_apply(Amazon)
if __name__ == "__main__":
    df2= pd.read_csv("az_val_review.csv")
    df2["clean_text"] = df2["text"].progress_apply(Amazon)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rushiraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100%|██████████| 5205/5205 [00:00<00:00, 53015.10it/s]


In [28]:
# Drop empty rows post-cleaning
df = df[df["clean_text"].str.len() > 0].copy()
df1 = df1[df1["clean_text"].str.len() > 0].copy()
df2 = df2[df2["clean_text"].str.len() > 0].copy()


df.to_csv("az_train_review_clean.csv", index=False)
print("Saved:", "az_train_review_clean.csv", "Shape:", df.shape)
df1.to_csv("az_test_review_clean.csv", index=False)
print("Saved:", "az_test_review_clean.csv", "Shape:", df1.shape)
df2.to_csv("az_val_review_clean.csv", index=False)
print("Saved:", "az_val_review_clean.csv", "Shape:", df2.shape)

Saved: az_train_review_clean.csv Shape: (31181, 3)
Saved: az_test_review_clean.csv Shape: (5198, 3)
Saved: az_val_review_clean.csv Shape: (5195, 3)


In [29]:
train_df=pd.read_csv("az_train_review_clean.csv")
test_df=pd.read_csv("az_test_review_clean.csv")
val_df=pd.read_csv("az_val_review_clean.csv")

In [30]:
pd.read_csv("az_train_review_clean.csv")

,text,target,clean_text
0,"Cooking microwave pizzas, yummy",2,"cooking microwave pizzas, yummy"
1,Any plans of allowing sub tasks to show up in ...,1,plans allowing sub tasks show widget?
2,"I love the humor, I just reworded it. Like sa...",2,"love humor, reworded it. like saying 'group th..."
3,naw idk what ur talkin about,1,naw idk ur talkin
4,That sucks to hear. I hate days like that,0,sucks hear. hate days like
...,...,...,...
31176,Grrrr....I got the wrong size coat for the sheep,0,grrrr....i got wrong size coat sheep
31177,4 cases of swine flu!,1,4 cases swine flu!
31178,excellent,2,excellent
31179,is sitting thru the boring bits in Titanic wai...,1,sitting thru boring bits titanic waiting good ...


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline

train_df = pd.read_csv("az_train_review_clean.csv")
test_df = pd.read_csv("az_test_review_clean.csv")
val_df = pd.read_csv("az_val_review_clean.csv")

X_train, y_train = train_df["clean_text"], train_df["target"]
X_test, y_test = test_df["clean_text"], test_df["target"]
X_val, y_val = val_df["clean_text"], val_df["target"]


pipe = Pipeline([
    ("tfidf", TfidfVectorizer(
        ngram_range=(1,3),  # capture bigrams
        min_df=5,
        max_df=0.9,
        sublinear_tf=True
    )),
    ("clf", LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        n_jobs=-1
    ))
])

pipe.fit(X_train, y_train)

print("Test report:")
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))
print("Confusion matrix (test):")
print(confusion_matrix(y_test, y_pred))

print("Validation report:")
y_val_pred = pipe.predict(X_val)
print(classification_report(y_val, y_val_pred, digits=4))
print("Confusion matrix (validation):")
print(confusion_matrix(y_val, y_val_pred))

Test report:
              precision    recall  f1-score   support

           0     0.6660    0.6790    0.6724      1545
           1     0.6123    0.6119    0.6121      1925
           2     0.7457    0.7332    0.7394      1728

    accuracy                         0.6722      5198
   macro avg     0.6747    0.6747    0.6747      5198
weighted avg     0.6726    0.6722    0.6724      5198

Confusion matrix (test):
[[1049  386  110]
 [ 425 1178  322]
 [ 101  360 1267]]
Validation report:
              precision    recall  f1-score   support

           0     0.6435    0.6957    0.6686      1515
           1     0.6070    0.5991    0.6030      1923
           2     0.7541    0.7120    0.7324      1757

    accuracy                         0.6654      5195
   macro avg     0.6682    0.6689    0.6680      5195
weighted avg     0.6674    0.6654    0.6659      5195

Confusion matrix (validation):
[[1054  364   97]
 [ 460 1152  311]
 [ 124  382 1251]]


In [5]:
print("Training data:", X_train.shape[0])
print("test data:",X_test.shape[0])
print("Validation data:",X_val.shape[0])

Training data: 31181
test data: 5198
Validation data: 5195


In [6]:
import pickle as pk
with open('pipe.pkl','wb')as f:
    pk.dump(pipe, f)